In [1]:
import os
import time
import tqdm
import numpy as np
import pandas as pd
import json
import itertools
import sys

sys.path.append(os.path.expanduser('~'))
from PpaPy.helper.io import makedir
from functions.read_write import NanConverter, load_data_from_keys

import warnings
warnings.filterwarnings(action='ignore', message='Mean of empty slice')

Please provide where your files are stored and where you would like your data to be saved in the following section.

In [8]:
datestr = time.strftime("%Y%m%d-%HH%MM")
inpath = "/gpfs/soma_fs/scratch/src/boeger/PpaPred_eren_35727184/ErenBoeger_2024" 
#inpath = '/gpfs/soma_fs/scratch/src/boeger/PpaPred_roca_35727184'

outpath = makedir(f"/gpfs/soma_fs/home/boeger/PpaPred/{os.path.basename(inpath)}")

# Eren Boeger 2024
#inpath_pattern = ["Exp1_WT_larvae", "Exp1_WT_OP50"]     
#inpath_pattern = ["Exp2_WT_larvae", "Exp2_tph1_larvae", "Exp2_cat2_larvae", "Exp2_tdc1_larvae", "Exp2_tbh1_larvae"]
#inpath_pattern = ["Exp3_tdc1_larvae", "Exp3_tyra2_larvae", "Exp3_ser2_larvae", "Exp3_tyra3_larvae", "Exp3_lgc55_larvae"]
#inpath_pattern = ["Exp3_ser3_larvae", "Exp3_ser6_larvae", "Exp3_octr1_larvae"]
inpath_pattern = ["Exp3_WT_larvae", "Exp3_tbh1_larvae",]
#inpath_pattern = ["Supp5_WT_larvae" "Supp5_tyramine_larvae", "Supp5_octopamine_larvae", "Supp4_tbh1tdc1_larvae", "Supp7_ser2tyra2tyra3_larvae"]

inpath_with_subfolders = True

overwrite = True

In [9]:
for data_str in inpath_pattern:
    print(data_str)
    ### I/O ################################################
    pattern_dir = [os.path.join(root, name) for root, dirs, files in os.walk(inpath) for name in dirs if data_str in name][0]
    all_files = [os.path.join(root, name) for root, dirs, files in os.walk(pattern_dir) for name in files if name.endswith('json') or name.endswith('csv') ]
    loc_all = {os.path.basename(f):f for f in all_files if 'prediction.json' in f}
    loc_summ = [f for f in all_files if 'summary.csv' in f]
    loc_trans =  [f for f in all_files if 'transitions.csv' in f]
    #loc_onoff = [f for f in all_files if 'onoff.json' in f]

    if outpath is None:
        outpath = os.path.commonpath(loc_all.values())
    JsonOut = os.path.join(outpath,f'{data_str}_batch.json')

    ### Load and save to batch json ##################################
    for fn,fpath in tqdm.tqdm(loc_all.items()):
        id = '_'.join(fn.split('_')[:-1])
        data = pd.read_json(fpath, orient='split')
        y = data['prediction']
        if 8 in np.unique(y):
            print(f"Warning: {fn}")
    
        proba = data.filter(regex='proba')
        idx = proba.columns.str.split('_', expand=True)
        proba.columns = idx
        mean_probas = {cl:np.nanmean(proba.loc[:,('proba',cl)][y == eval(cl)]) for cl in proba.columns.levels[1]}
        
        summ_ = pd.read_csv([l for l in loc_summ if id in l][0])
        
        fr_transition_ = pd.read_csv([l for l in loc_trans if id in l][0], index_col=0)
        fr_transition_[fr_transition_==0] = np.nan # for now until processing in FeedingPrediction is fixed
        fr_transition_tuple = dict(zip(str(list(itertools.product(fr_transition_.columns.astype(int), fr_transition_.index))).strip('[()]').split('), ('), fr_transition_.values.T.flatten()))

        data_mean = data[['velocity', 'rate', 'prediction']].groupby('prediction').mean().reindex(range(-1,8))
        
        # prep of json file structure
        etho = {id:{'count':summ_.duration_count.fillna(0).to_dict(),
                    'mean duration':summ_.duration_mean.to_dict(),
                    'rel time in': summ_.duration_relative.fillna(0).to_dict(),
                    'mean velocity': data_mean.velocity.to_dict(),
                    'mean rate': data_mean.rate.to_dict(),
                    'mean transitions':fr_transition_tuple,
                    'mean prediction probability': mean_probas,
                    'ethogram':y.to_list()}}
        
        # if file exists and overwrite is false
        ow_org = overwrite
        if os.path.isfile(JsonOut) and not overwrite:
            with open(JsonOut, "r") as jsonfile:
                batch = json.load(jsonfile)
        else:
            batch = {}
            overwrite = False
        
        batch.update(etho)
        jsnF = json.dumps(batch, indent = 4, cls=NanConverter)
        with open(JsonOut, "w") as outfile:
            outfile.write(jsnF)

    overwrite = ow_org

Exp3_WT_larvae


100%|██████████| 170/170 [02:35<00:00,  1.09it/s]


Exp3_tbh1_larvae


100%|██████████| 197/197 [02:30<00:00,  1.31it/s]


In [6]:
[os.path.join(root, name) for root, dirs, files in os.walk(inpath) for name in dirs if data_str in name]

['/gpfs/soma_fs/scratch/src/boeger/PpaPred_eren_35727184/Exp2_WT_larvae_discarded']

In [7]:
next(os.walk(inpath))

('/gpfs/soma_fs/scratch/src/boeger/PpaPred_eren_35727184',
 ['Exp2_cat2_larvae',
  'Exp2_tbh1_larvae',
  'Exp2_tph1_larvae',
  'Pharma_octopamine_larvae',
  'Exp2_WT_larvae_discarded',
  'Exp3_ser2_larvae',
  'Exp2_tdc1_larvae',
  'Exp3_tdc1_larvae',
  'Exp3_ser6_larvae',
  'Pharma_tyramine_larvae',
  'Exp3_tyra2_larvae',
  'Exp3_tbh1_larvae',
  'Exp1_WT_OP50',
  'Exp3_ser3_larvae',
  'Exp3_tyra3_larvae',
  'Exp3_tbh1tdc1_larvae',
  'Exp3_lgc55_larvae',
  'Exp1_WT_larvae',
  'Exp3_WT_larvae',
  'Exp3_octr1_larvae'],
 ['20240515-16H10M_PpaForagingPrediction.log',
  '39952187_20240617-12H22M_PpaForagingPrediction.log',
  '36549508_20240523-10H51M_PpaForagingPrediction.log',
  '36534869_20240523-09H22M_PpaForagingPrediction.log',
  '36553226_20240523-11H06M_PpaForagingPrediction.log',
  '36534870_20240523-09H22M_PpaForagingPrediction.log',
  '36391051_20240522-11H18M_PpaForagingPrediction.log',
  '39952495_20240617-12H44M_PpaForagingPrediction.log',
  '36555248_20240523-11H54M_PpaForaging